# [Petunjuk Penyelesaian Project](https://academy.dqlab.id/main/projectcode/16/109/516?pr=0)

Project ini terdiri dari 3 soal, yaitu:

- Mendapatkan insight top 10 dan bottom 10 dari produk yang terjual.
- Mendapatkan daftar seluruh kombinasi paket produk dengan korelasi yang kuat.
- Mendapatkan daftar seluruh kombinasi paket produk dengan item tertentu.

In [18]:
install.packages("arules")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [19]:
library(arules)
transaksi_tabular <- read.transactions(file="https://storage.googleapis.com/dqlab-dataset/transaksi_dqlab_retail.tsv", 
                                       format="single", 
                                       sep="\t", 
                                       cols=c(1,2), 
                                       skip=1)
write(transaksi_tabular, file="test_project_retail_1.txt", sep=",")

# [Output Awal: Statistik Top 10](https://academy.dqlab.id/main/projectcode/16/109/517?pr=)

In [20]:
library(arules)

data <- read.transactions(file = "transaksi_dqlab_retail.tsv",
                          format = "single",
                          sep = "\t",
                          cols = c(1,2),
                          skip = 1)
top_10 <- sort(itemFrequency(data, type = "absolute"),
               decreasing = TRUE)[1:10]
hasil <- data.frame("Nama Produk" = names(top_10),
                    "Jumlah" = top_10, 
                    row.names = NULL)
write.csv(hasil, file = "top10_item_retail.txt")

# [Output Awal: Statistik Bottom 10](https://academy.dqlab.id/main/projectcode/16/109/518?pr=)

In [21]:
library(arules)

data <- read.transactions(file = "transaksi_dqlab_retail.tsv",
                          format = "single",
                          sep = "\t",
                          cols = c(1,2),
                          skip = 1)
bottom_10 <- sort(itemFrequency(data,type = "absolute"),
                  decreasing = FALSE)[1:10]
hasil <- data.frame("Nama Produk" = names(bottom_10),
                    "Jumlah" = bottom_10,
                    row.names = NULL)
write.csv(hasil,file = "bottom10_item_retail.txt")

# [Mendapatkan Kombinasi Produk yang menarik](https://academy.dqlab.id/main/projectcode/16/109/519?pr=)

Setelah yakin Anda dapat melakukannya Pak Agus ingin Anda mengirimkan file yang berisi daftar 10 paket kombinasi produk yang paling "menarik".

Anda pertamanya bingung, apa sih definisi menarik versi Pak Agus ini. Setelah wawancara intensif, ternyata pengertiannya adalah sebagai berikut:

- Memiliki asosiasi atau hubungan erat.
- Kombinasi produk minimal 2 item, dan maksimum 3 item.
- Kombinasi produk itu muncul setidaknya 10 dari dari seluruh transaksi.
- Memiliki tingkat confidence minimal 50 persen.

In [22]:
library(arules)

nama_file <- "transaksi_dqlab_retail.tsv"
transaksi_tabular <- read.transactions(file = nama_file, 
                                       format = "single",
                                       sep = "\t",
                                       cols = c(1,2),
                                       skip = 1)
apriori_rules <- apriori(transaksi_tabular,
                         parameter = list(supp = 10/length(transaksi_tabular),
                                          conf = 0.5,
                                          minlen = 2,
                                          maxlen=3))
apriori_rules <- head(sort(apriori_rules, by = "lift",decreasing = T),n = 10)
inspect(apriori_rules)
write(apriori_rules, file="kombinasi_retail.txt")

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime     support minlen
        0.5    0.1    1 none FALSE            TRUE       5 0.002898551      2
 maxlen target  ext
      3  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 10 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[69 item(s), 3450 transaction(s)] done [0.00s].
sorting and recoding items ... [68 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 3

Warning message in apriori(transaksi_tabular, parameter = list(supp = 10/length(transaksi_tabular), :
“Mining stopped (maxlen reached). Only patterns up to a length of 3 returned!”


 done [0.01s].
writing ... [4637 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].
     lhs                              rhs                              support confidence    coverage     lift count
[1]  {Tas Makeup,                                                                                                   
      Tas Pinggang Wanita}         => {Baju Renang Anak Perempuan} 0.010434783  0.8780488 0.011884058 24.42958    36
[2]  {Tas Makeup,                                                                                                   
      Tas Travel}                  => {Baju Renang Anak Perempuan} 0.010144928  0.8139535 0.012463768 22.64629    35
[3]  {Tas Makeup,                                                                                                   
      Tas Ransel Mini}             => {Baju Renang Anak Perempuan} 0.011304348  0.7358491 0.015362319 20.47322    39
[4]  {Sunblock Cream,                                                                  

# [Mencari Paket Produk yang bisa dipasangkan dengan Item Slow-Moving](https://academy.dqlab.id/main/projectcode/16/109/520?pr=)

**Slow-moving item** adalah produk yang pergerakan penjualannya lambat atau kurang cepat. Ini akan bermasalah apabila item produk tersebut masih menumpuk.

Kadang kala item ini belum tentu tidak laku, hanya saja mungkin harganya tidak bagus dan jarang dibutuhkan jika dijual satuan.  Nah, jika tidak dijual satuan kita perlu cari asosiasi kuat dari item produk ini dengan produk lain sehingga jika dipaketkan akan menjadi lebih menarik.

Pak Agus juga meyakini hal ini, dan ingin agar Anda membantu mengidentifikasi dua item produk yang menurut dia stoknya masih banyak dan perlu dicari pasangan item untuk pemaketannya.

Dua item produk tersebut adalah "Tas Makeup" dan "Baju Renang Pria Anak-anak". Pak Agus ingin meminta kombinasi yang bisa dipaketkan dengan kedua produk tersebut.

Masing-masing produk tersebut dikeluarkan 3 rules yang asosiasinya paling kuat, sehingga total ada 6 rules. Persyaratan-persyaratan asosiasi kuat ini masih sama dengan yang telah disebutkan Pak Agus sebelumnya, kecuali tingkat confidence dicoba pada tingkat minimal 0.1.

In [23]:
library(arules)

nama_file <- "transaksi_dqlab_retail.tsv"
transaksi_tabular <- read.transactions(file = nama_file,
                                       format = "single",
                                       sep = "\t",
                                       cols = c(1,2),
                                       skip = 1)
jumlah_transaksi <- length(transaksi_tabular)
jumlah_kemunculan_minimal <- 10
apriori_rules <- apriori(transaksi_tabular,
                         parameter = list(supp = jumlah_kemunculan_minimal/jumlah_transaksi,
                                          conf = 0.1,
                                          minlen = 2,
                                          maxlen = 3))

# Filter
apriori_rules1 <- subset(apriori_rules, lift > 1 & rhs %in% "Tas Makeup")
apriori_rules1 <- sort(apriori_rules1, by = "lift", decreasing = T)[1:3]
apriori_rules2 <- subset(apriori_rules, lift > 1 & rhs %in% "Baju Renang Pria Anak-anak")
apriori_rules2 <- sort(apriori_rules2, by = "lift", decreasing = T)[1:3]
apriori_rules <- c(apriori_rules1, apriori_rules2)
inspect(apriori_rules)
write(apriori_rules, file = "kombinasi_retail_slow_moving.txt")

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime     support minlen
        0.1    0.1    1 none FALSE            TRUE       5 0.002898551      2
 maxlen target  ext
      3  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 10 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[69 item(s), 3450 transaction(s)] done [0.00s].
sorting and recoding items ... [68 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 3

Warning message in apriori(transaksi_tabular, parameter = list(supp = jumlah_kemunculan_minimal/jumlah_transaksi, :
“Mining stopped (maxlen reached). Only patterns up to a length of 3 returned!”


 done [0.01s].
writing ... [39832 rule(s)] done [0.01s].
creating S4 object  ... done [0.01s].
    lhs                              rhs                              support confidence   coverage     lift count
[1] {Baju Renang Anak Perempuan,                                                                                  
     Tas Pinggang Wanita}         => {Tas Makeup}                 0.010434783  0.8000000 0.01304348 19.57447    36
[2] {Baju Renang Anak Perempuan,                                                                                  
     Tas Ransel Mini}             => {Tas Makeup}                 0.011304348  0.7959184 0.01420290 19.47460    39
[3] {Baju Renang Anak Perempuan,                                                                                  
     Celana Pendek Green/Hijau}   => {Tas Makeup}                 0.010144928  0.7777778 0.01304348 19.03073    35
[4] {Gembok Koper,                                                                                  